<img src="https://img.icons8.com/bubbles/100/000000/3d-glasses.png" style="height:50px;display:inline"> University of Haifa - Computer Vision course


## Homework 2 - 3D Vision
---

### <a style='color:red'> Due Date: Thursday 12.6.2025 </a>

### <img src="https://img.icons8.com/bubbles/50/000000/upload-to-cloud.png" style="height:50px;display:inline"> Submission Guidelines
---
#### READ THIS CAREFULLY
* Submission only in **pairs**, unless you have already got an approval to work on your own

* Submission is only through Moodle

* You can choose your working environment:
    * `Jupyter Notebook`, locally with <a href="https://www.anaconda.com/distribution/">Anaconda</a> or online on <a href="https://colab.research.google.com/">Google Colab</a>
        * Colab also supports running code on GPU, but you will not need it for this excercise.
    * Python IDE such as <a href="https://www.jetbrains.com/pycharm/">PyCharm</a> or <a href="https://code.visualstudio.com/">Visual Studio Code</a>.
        * Both allow editing and running Jupyter Notebooks.
<div style="margin-top:6px;"></div>

* You should submit only this ipynb notebook file, **with all cells executed such that the results will be included**.
<div style="margin-top:6px;"></div>

* Read the instructions carefully:
    * You have functions to complete and scripts to write - look for <span style="color:red">''' YOUR CODE HERE '''</span>
        * Make sure to properly document your code
        * Notice where you can use existing implementations (e.g. opencv) freely and in which cases you have to provide your own implementation of certain steps
    * You have some explanations and discussions to write - look for <span style="color: red; font-weight: bold;">YOUR EXPLANATION\DISCUSSION HERE</span>
        * Make sure to answer shortly but informatively
<div style="margin-top:6px;"></div>

* Code of Honor:
    * We wish to avoid dealing with copied homework. However, we will not hesitate to take serious actions against students that are caught with violations of the Honor Code. Note that in this course, it is rather easy to detect similar submissions, as most of them require your "personal" touch.     
<div style="margin-top:6px;"></div>

* Submission date:
    * Submission date of 12.6.2025 is final and will not be delayed
    * However, you may submit up to 2 days late (till Sunday 14.6.2025) at the cost of 4 points per day
    * Any (fully justified and documented) requests for delay must be sent till Sunday 8.6 at the latest
    * The excercise is long and isn't easy - **make sure to start early**


### <img src="https://img.icons8.com/dusk/64/000000/python.png" style="height:50px;display:inline"> Python Libraries
---

* `numpy`
* `matplotlib`
* `opencv` (or `scikit-image`)
* `scikit-learn`
* `scipy`
* Anything else you need (`os`, `pandas`, `csv`, `json`,...)

# <font color="red">IMPORTANT:</font> Make sure that each output image has a title to it.

# Part 1 - Constraining putative matches using geometry
---

### Implement the functions below (separately), given a pair of images to be matched:

## 1. Obtain SIFT-based matches
* Compute SIFT features in each of the images
* Perform SIFT matching between the two sets of features, to obtain a set of matches M

## 2. Visualize a set of matches
* Visualize the image pair, side-by-side, with thin lines connecting matching points (You may visualize a large random subset, if there are too many matches and the result is too cluttered)

## 3. Recover the Epipolar geometry
* Given the set of matches, estimate a Fundamental matrix F

## 4. Visualize the Epipolar geometry
* Given F, sample 10 2D points, uniformly at random, in image 1 and compute (using F), the epipolar lines in image 2.
* Visualize the image pair, side-by-side. Choose 10 unique different colors (preferably spread uniformly over some colorspace) and then use colored markers to overlay the points in image 1 and the corresponding colors to overlay the epipolar lines in image 2.
* Repeat the above, by switching roles (random points in image 2 and matching epipoles in image 1).

## 5. Constrain a set of matches, by filtering them with the obtained geometry
* Find and return the subset M' of the original set of matches M, that 'agree' with the obtained F, up to some error threshold.


### Using the above functions, for each pair of images in data/pairs_to_match, perform the following:
* Visualize a simple SIFT-based set of matches
* Estimate the fundamental matrix F and visualize the epipolar geometry
* Constrain the set of matches using F and visualize the result.
* Do you observe that constraining the matches to the epipolar geometry (given by F) is helpful in any way? (Was it able to filter out all of the outliers? Did it filter out any inliers?). Explain in the second cell below.



In [ ]:
# ALL OF YOUR CODE FOR PART 1 SHOULD BE IN THIS CELL

# REQUIRED OUTPUTS (For each of the input stereo pairs in data/pairs_to_match):
# 1. Visualize matches
# 2. Visualize epipolar geometry
# 3. Visualize matches after constraints

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Function to load image
def load_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Image not found at {image_path}")
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # for matplotlib display
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # for SIFT
    return img_rgb, img_gray

# Function for SIFT matching
def compute_sift_matches(img1_gray, img2_gray):
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1_gray, None)
    kp2, des2 = sift.detectAndCompute(img2_gray, None)

    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False) # crossCheck=False for ratio test
    raw_matches = bf.knnMatch(des1, des2, k=2)

    good_matches = []
    # Lowe's ratio test
    for m, n in raw_matches:
        if m.distance < 0.75 * n.distance: 
            good_matches.append(m)
    return kp1, kp2, good_matches

# Function to visualize matches
def visualize_matches(img1_rgb, kp1, img2_rgb, kp2, matches, title="SIFT Matches"):
    # Limit matches to display if too many
    display_matches = matches
    if len(matches) > 100:
        # Ensure that the chosen matches are of type cv2.DMatch if drawMatches expects that.
        # np.random.choice returns an ndarray, so convert it to a list of DMatch objects.
        chosen_indices = np.random.choice(len(matches), 100, replace=False)
        display_matches = [matches[i] for i in chosen_indices]

    # Create a new image by drawing matches
    # Ensure kp1 and kp2 are lists of KeyPoint objects
    match_img = cv2.drawMatches(img1_rgb, kp1, img2_rgb, kp2, display_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    plt.figure(figsize=(20, 10))
    plt.imshow(match_img)
    plt.title(title)
    plt.axis('off') # Hide axes ticks and labels
    plt.show()

# Function to estimate Fundamental Matrix
def estimate_fundamental_matrix(kp1, kp2, matches):
    if not matches or len(matches) < 7: # Need at least 7 points for FM_RANSAC or FM_8POINT
        print("Not enough matches to estimate Fundamental Matrix.")
        return None, None
    pts1 = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    pts2 = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC, ransacReprojThreshold=3.0, confidence=0.99)
    # ransacReprojThreshold: Maximum distance from a point to an epipolar line in pixels, above which the point is considered an outlier.
    # confidence: Desired confidence level (probability) that the estimated matrix is correct.
    if F is None or F.shape != (3,3):
        print("Fundamental Matrix estimation failed or returned invalid shape.")
        return None, None
    return F, mask

# Helper function to draw epilines
def draw_epilines_helper(img_lines_on, img_pts_on, lines, pts, colors):
    r, c = img_lines_on.shape[:2]
    img_lines_on_copy = img_lines_on.copy()
    img_pts_on_copy = img_pts_on.copy()
    for i in range(len(lines)):
        line = lines[i].ravel() # line is [[a,b,c]]
        pt = pts[i].ravel() # pt is [[x,y]]
        color = tuple(colors[i].tolist()) # Convert numpy array color to tuple
        
        # Epipolar line equation: ax + by + c = 0
        # y = (-c - ax) / b
        # Check for horizontal lines (b close to 0) to avoid division by zero or very large y values
        if abs(line[1]) > 1e-5: # if b is not close to zero
            x0, y0 = map(int, [0, -line[2]/line[1] ])
            x1, y1 = map(int, [c, -(line[2]+line[0]*c)/line[1] ])
        else: # Horizontal line, y = -c/b. If b is zero, it's a vertical line x = -c/a
             # For simplicity in visualization, draw a line across x at a fixed y if b is tiny, or vertical if a is tiny.
            if abs(line[0]) > 1e-5: # Vertical line x = -c/a
                x0, y0 = map(int, [-line[2]/line[0], 0])
                x1, y1 = map(int, [-line[2]/line[0], r])
            else: # Line is degenerate or too complex for this simple drawing, skip or handle as error
                # This case should ideally not happen with valid F and points
                continue
        img_lines_on_copy = cv2.line(img_lines_on_copy, (x0,y0), (x1,y1), color, 1)
        pt_int = tuple(map(int, pt)) 
        img_pts_on_copy = cv2.circle(img_pts_on_copy, pt_int, 5, color, -1)
    return img_lines_on_copy, img_pts_on_copy

# Function to visualize epipolar geometry
def visualize_epipolar_geometry(img1_rgb, img2_rgb, pts_sampled, F, is_pts_in_img1=True, title="Epipolar Geometry"):
    num_pts = pts_sampled.shape[0]
    # Generate unique colors for points and lines
    # Using a colormap like 'viridis' or 'jet' from matplotlib to get distinct colors
    cmap = plt.get_cmap('jet', num_pts)
    colors = [cmap(i)[:3] for i in range(num_pts)] # Get RGB, ignore alpha
    colors = (np.array(colors) * 255).astype(np.uint8) # Convert to 0-255 range for OpenCV

    img1_display = img1_rgb.copy()
    img2_display = img2_rgb.copy()

    if is_pts_in_img1:
        # Points are in img1, compute epilines in img2
        lines_in_img2 = cv2.computeCorrespondEpilines(pts_sampled.reshape(-1, 1, 2), 1, F)
        img2_display, img1_display = draw_epilines_helper(img2_display, img1_display, lines_in_img2, pts_sampled, colors)
        # pts_sampled are for img1, lines are for img2
    else:
        # Points are in img2, compute epilines in img1
        lines_in_img1 = cv2.computeCorrespondEpilines(pts_sampled.reshape(-1, 1, 2), 2, F)
        img1_display, img2_display = draw_epilines_helper(img1_display, img2_display, lines_in_img1, pts_sampled, colors)
        # pts_sampled are for img2, lines are for img1

    # Combine images side-by-side
    combined_img = np.concatenate((img1_display, img2_display), axis=1)

    plt.figure(figsize=(20, 10))
    plt.imshow(combined_img)
    plt.title(title)
    plt.axis('off')
    plt.show()


# --- Main processing loop for Part 1 ---
image_dir = "data/pairs_to_match/"
image_pairs = [
    ("graf1.jpg", "graf2.jpg"),
    ("object0019.view01.jpg", "object0019.view02.jpg"),
    ("object0050.view01.jpg", "object0050.view02.jpg"),
    ("wall1.jpg", "wall3.jpg")
]

print("Starting Part 1: SIFT Matching and Visualization\n")

for img_name1, img_name2 in image_pairs:
    print(f"Processing pair: {img_name1} and {img_name2}")
    img1_path = os.path.join(image_dir, img_name1)
    img2_path = os.path.join(image_dir, img_name2)

    try:
        img1_rgb, img1_gray = load_image(img1_path)
        img2_rgb, img2_gray = load_image(img2_path)

        # 1. Obtain SIFT-based matches
        kp1, kp2, matches = compute_sift_matches(img1_gray, img2_gray)
        print(f"Found {len(matches)} good SIFT matches for {img_name1}-{img_name2}.")

        # 2. Visualize a set of matches
        sift_matches_title = f"Initial SIFT Matches: {os.path.splitext(img_name1)[0]} and {os.path.splitext(img_name2)[0]}"
        visualize_matches(img1_rgb, kp1, img2_rgb, kp2, matches, title=sift_matches_title)

        # 3. Recover the Epipolar geometry
        if len(matches) >= 7: # Minimum number of points for findFundamentalMat
            F, f_mask = estimate_fundamental_matrix(kp1, kp2, matches)
            if F is not None:
                print(f"Fundamental Matrix estimated for {img_name1}-{img_name2}.")
                # Prepare points for epipolar line visualization
                # Get points from original matches list, not the potentially filtered 'good_matches' from F estimation's mask yet
                # as we want to sample from all SIFT matches for visualization of epipolar lines.
                pts_for_viz_img1 = np.float32([kp1[m.queryIdx].pt for m in matches])
                pts_for_viz_img2 = np.float32([kp2[m.trainIdx].pt for m in matches])

                # 4. Visualize the Epipolar geometry
                # Sample 10 points from image 1
                if len(pts_for_viz_img1) >= 10:
                    sample_indices1 = np.random.choice(pts_for_viz_img1.shape[0], 10, replace=False)
                    pts1_sampled = pts_for_viz_img1[sample_indices1]
                    epipolar_title1 = f"Epipolar Geometry (Points from {img_name1}, Lines on {img_name2})"
                    visualize_epipolar_geometry(img1_rgb, img2_rgb, pts1_sampled, F, is_pts_in_img1=True, title=epipolar_title1)
                else:
                    print(f"Not enough points in image 1 ({len(pts_for_viz_img1)}) to sample 10 for epipolar visualization.")

                # Sample 10 points from image 2
                if len(pts_for_viz_img2) >= 10:
                    sample_indices2 = np.random.choice(pts_for_viz_img2.shape[0], 10, replace=False)
                    pts2_sampled = pts_for_viz_img2[sample_indices2]
                    epipolar_title2 = f"Epipolar Geometry (Points from {img_name2}, Lines on {img_name1})"
                    visualize_epipolar_geometry(img1_rgb, img2_rgb, pts2_sampled, F, is_pts_in_img1=False, title=epipolar_title2)
                else:
                    print(f"Not enough points in image 2 ({len(pts_for_viz_img2)}) to sample 10 for epipolar visualization.")

                # 5. Constrain a set of matches using F and visualize the result
                if f_mask is not None:
                    constrained_matches = [m for i, m in enumerate(matches) if f_mask[i].item() == 1]
                    print(f"Number of matches after constraining with F: {len(constrained_matches)}")
                    constrained_matches_title = f"Constrained SIFT Matches: {os.path.splitext(img_name1)[0]} and {os.path.splitext(img_name2)[0]}"
                    visualize_matches(img1_rgb, kp1, img2_rgb, kp2, constrained_matches, title=constrained_matches_title)
                else:
                    print("No inlier mask available to constrain matches.")
            else:
                print(f"Could not estimate Fundamental Matrix for {img_name1}-{img_name2}. Skipping epipolar geometry and constrained matches visualization.")
        else:
            print(f"Not enough SIFT matches ({len(matches)}) to estimate F for {img_name1}-{img_name2}. Skipping epipolar geometry and constrained matches visualization.")

        print(f"Finished processing {img_name1} and {img_name2}.")
    except FileNotFoundError as e:
        print(f"Error: {e}. Skipping this pair.")
    except Exception as e:
        print(f"An unexpected error occurred while processing {img_name1} and {img_name2}: {e}. Skipping this pair.")
    print("---")

print("\nFinished Part 1.")

<span style="color: red; font-weight: bold;">YOUR EXPLANATION HERE</span>

**Observations on Constraining Matches with Epipolar Geometry:**

*   **Helpfulness of Epipolar Constraint:**
    *   Discuss whether constraining matches using the Fundamental matrix (F) was generally helpful. (e.g., "Yes, constraining matches using the epipolar geometry proved to be quite helpful in refining the initial set of SIFT matches.")
    *   Comment on the visual difference between the initial set of matches and the constrained set. (e.g., "Visually, the set of constrained matches appeared much cleaner and more geometrically consistent across the image pairs compared to the raw SIFT matches.")

*   **Filtering Outliers:**
    *   For each image pair (or generally), describe if the method was successful in filtering out outliers (incorrect matches). (e.g., "For most image pairs, especially `graf` and `wall`, the F-matrix constraint successfully filtered out a significant number of outlier matches. These outliers were often visible as lines crossing in unrealistic ways in the initial match visualization.")
    *   Mention if *all* outliers were removed or if some persisted. (e.g., "While a majority of outliers were removed, some subtle incorrect matches might have remained, particularly if they coincidentally aligned somewhat with the epipolar geometry or if the scene had repetitive textures.")

*   **Filtering Inliers (Accidental Removal of Correct Matches):**
    *   Discuss whether the geometric constraint accidentally filtered out any valid inliers (correct matches). (e.g., "In general, the method seemed to preserve most of the true inliers. However, there might have been a few instances where valid matches were discarded, possibly due to slight inaccuracies in keypoint localization or a very strict threshold in the RANSAC estimation of F.")
    *   Consider factors that might lead to inlier rejection (e.g., very small baselines, errors in F estimation, non-rigid scene parts if any).

*   **Specific Image Pair Examples (Optional but good):**
    *   If specific behaviors were noted for certain image pairs, mention them. (e.g., "The `object` pairs, which might have less texture or smaller baselines, perhaps showed a different ratio of outliers to inliers compared to the `graf` pair.")

*   **Overall Assessment:**
    *   Conclude with an overall assessment of the technique's effectiveness for the given datasets. (e.g., "Overall, applying epipolar constraints is a crucial step in improving the quality of feature matches for tasks like 3D reconstruction or structure from motion, as it effectively removes mismatches that are inconsistent with a rigid scene geometry.")

**Note to Student:** Please elaborate on these points based on the actual visualizations produced by your code. Quantify if possible (e.g., "reduced number of matches from X to Y for graf1/graf2 pair").

# Part 2 - Middle-View synthesis from a rectified stereo pair.
---

### In this section, given a *rectified* image pair, we wish to generate/synthesize the image that would have been captured by a camera situated exactly in the middle, between the pair of poses.

## We will follow two different strategies to do so:
* Strategy A: Via photometric stereo
* Strategy B: Via sparse reconstruction

## Details for Strategy A:
* Compute a (dense) disparity map between the pair of images: **Implement your own (don't use a ready library function for this)**
* Use this map to synthesize the 'central' image
* Explain what problems arise in this approach and how your solution tries to deal with them

## Details for Strategy B:
* Use the code from the previous parts to obtain a set of matches - ones that obeys with the geometry. How would you do this even without knowing the fundamental matrix?
* Using triangulation, find the locations of the interest points in the new 'central' image (In practice - this can be done by simply using the disparity for each match). At this point - You should have 'triplet' matches, each consisting of 1 point in each of the 3 viewpoints.
* Take the set of (match) points in one of the images (left, right or middle - to your choice) and sub-divide the image into small triangular or quadrilateral sub-regions, whose vertices are placed on the matched interest points. This can be done, for example, by computing any kind of triangulation over the interest points (see the 'Dylan' example under the 'Examples' folder). Once obtaining the subdivision (e.g. triangulation), it can be transfered to the other two images, using the corresponding matching points.
* For each region (triangle or quadrilateral), copy the contents from one of the input images to the target (middle) image, using either an affine transformation (for a triangle) or a homography (for a quadrilateral).
* Explain what problems arise in this approach and how your solution tries to deal with them.

## Implement your solutions and expain them in the following 4 cells.


In [ ]:
# ALL OF YOUR CODE FOR STATEGY A SHOULD BE IN THIS CELL

# REQUIRED OUTPUTS (For each of the input stereo pairs in data/rectified_pairs):
# 1. Visualize, side-by-side: left-image | right-image
# 2. Visualize, side-by-side: left-disparity-image | right_disparity-image
# 3. Visualize, by constructing an infinite-loop GIF of the four images: img_1->img_mid->img_2->img_mid->,
#    using the following given function:

import imageio
from IPython.display import Image

def create_and_display_gif(image_files, gif_path='animation.gif', duration=0.5):
    """
    Creates a GIF from a list of image files and displays it in a Jupyter notebook.

    Parameters:
    - image_files (list of str): Paths to the input image files.
    - gif_path (str): Path where the output GIF will be saved.
    - duration (float): Duration per frame in seconds.

    Returns:
    - IPython.display.Image object displaying the GIF.
    """
    with imageio.get_writer(gif_path, mode='I', duration=duration) as writer:
        for filename in image_files:
            image = imageio.imread(filename)
            writer.append_data(image)

    return Image(filename=gif_path)

# example usage:
# image_files = ['frame_1.jpg', 'frame_middle.jpg', 'frame_2.jpg', 'frame_middle.jpg'] 
# create_and_display_gif(image_files, gif_path='my_gif.gif', duration=0.5)

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import imageio # Already imported above but good to have for clarity
from IPython.display import Image # Already imported above

def compute_dense_disparity_sad(img_left_gray, img_right_gray, max_disparity, block_size):
    """Computes dense disparity map using Sum of Absolute Differences (SAD)."""
    h, w = img_left_gray.shape
    disparity_map = np.zeros((h, w), dtype=np.float32)
    half_block = block_size // 2

    # Pad images to handle borders
    img_left_pad = cv2.copyMakeBorder(img_left_gray, half_block, half_block, half_block, half_block, cv2.BORDER_REFLECT)
    img_right_pad = cv2.copyMakeBorder(img_right_gray, half_block, half_block, half_block, half_block, cv2.BORDER_REFLECT)

    for y in range(half_block, h + half_block):
        for x in range(half_block, w + half_block):
            best_disparity = 0
            min_sad = float('inf')

            # Define the block in the left image
            block_left = img_left_pad[y - half_block : y + half_block + 1, x - half_block : x + half_block + 1]

            # Search for the best match in the right image (along the epipolar line, which is the same row for rectified images)
            for d in range(max_disparity + 1):
                if (x - d - half_block) >= 0 and (x - d + half_block + 1) <= img_right_pad.shape[1]:
                    block_right = img_right_pad[y - half_block : y + half_block + 1, x - d - half_block : x - d + half_block + 1]
                    if block_left.shape == block_right.shape:
                        sad = np.sum(np.abs(block_left.astype(np.float32) - block_right.astype(np.float32)))
                        if sad < min_sad:
                            min_sad = sad
                            best_disparity = d
            disparity_map[y - half_block, x - half_block] = best_disparity
    return disparity_map

def synthesize_middle_view(img_left_rgb, img_right_rgb, disparity_map_left, disparity_map_right=None):
    """Synthesizes the middle view using forward warping with disparity maps."""
    h, w, _ = img_left_rgb.shape
    middle_view = np.zeros_like(img_left_rgb, dtype=np.float32)
    count_map = np.zeros((h,w), dtype=np.float32) # To count contributions for averaging

    # Forward warp from left image
    for y in range(h):
        for x in range(w):
            disparity = disparity_map_left[y, x]
            # Middle view x-coordinate: xl - d/2. Since image is rectified, y_mid = y_left
            # xl is current x. We shift it left by d/2.
            x_mid = int(round(x - disparity / 2.0))
            if 0 <= x_mid < w:
                middle_view[y, x_mid] += img_left_rgb[y, x]
                count_map[y, x_mid] += 1

    # If disparity_map_right is provided, forward warp from right image
    # This part assumes disparity_map_right gives positive disparities for points shifting *leftwards* from right to left image
    # So, a point (xr, yr) in right image corresponds to (xr + dr, yr) in left image.
    # The middle point is xr + dr/2.
    if disparity_map_right is not None:
        for y in range(h):
            for x in range(w):
                disparity_r = disparity_map_right[y, x] # Positive disparity means shift this much left to match left image
                # Middle view x-coordinate: xr + dr/2
                x_mid = int(round(x + disparity_r / 2.0))
                if 0 <= x_mid < w:
                    middle_view[y, x_mid] += img_right_rgb[y, x]
                    count_map[y, x_mid] += 1
    
    # Average the contributions
    # Avoid division by zero for pixels with no contribution
    middle_view_avg = np.zeros_like(middle_view, dtype=np.uint8)
    for c in range(3): # For each color channel
      middle_view_avg[:,:,c] = np.divide(middle_view[:,:,c], count_map, out=np.zeros_like(middle_view[:,:,c], dtype=np.float32), where=count_map!=0)
    
    # Basic hole filling (e.g., simple horizontal blur or using a more sophisticated inpainting method)
    # For now, let's use a simple approach: if a pixel has no contribution, take from neighbor or leave black
    # A more robust solution would be better for real applications.
    # For pixels where count_map is 0, middle_view_avg will be 0. We can try a simple horizontal fill.
    for y in range(h):
        for x in range(w):
            if count_map[y,x] == 0:
                # Simplistic hole filling: copy from left neighbor if available
                if x > 0 and count_map[y, x-1] > 0:
                    middle_view_avg[y,x] = middle_view_avg[y,x-1]
                # else, it remains black or could try right neighbor, etc.

    return middle_view_avg.astype(np.uint8)


# --- Main processing loop for Part 2, Strategy A ---
rectified_image_dir = "data/rectified_pairs/"
# Pairs could be ('image_1.jpg', 'image_2.jpg'), ('scene_1.jpg', 'scene_2.jpg') etc.
# For this example, let's assume we have one pair to process explicitly by name.
# The notebook implies processing for *each* pair in data/rectified_pairs
rectified_pairs = [
    ("image_1.jpg", "image_2.jpg"),
    ("scene_1.jpg", "scene_2.jpg") 
    # Add more pairs here if they exist in the folder
]

print("Starting Part 2, Strategy A: Middle-View Synthesis via Photometric Stereo\n")

output_dir_gifs = "output_gifs_strategy_A"
if not os.path.exists(output_dir_gifs):
    os.makedirs(output_dir_gifs)

temp_image_files_for_gif = [
    os.path.join(output_dir_gifs, "_temp_frame_1.png"),
    os.path.join(output_dir_gifs, "_temp_frame_mid.png"),
    os.path.join(output_dir_gifs, "_temp_frame_2.png"),
    os.path.join(output_dir_gifs, "_temp_frame_mid.png") # Mid frame repeated for GIF sequence
]

for img_name_left, img_name_right in rectified_pairs:
    print(f"Processing rectified pair: {img_name_left} and {img_name_right}")
    img_left_path = os.path.join(rectified_image_dir, img_name_left)
    img_right_path = os.path.join(rectified_image_dir, img_name_right)

    try:
        img_left_rgb, img_left_gray = load_image(img_left_path) # load_image should be defined in Part 1 or here
        img_right_rgb, img_right_gray = load_image(img_right_path)

        # 1. Visualize side-by-side: left-image | right-image
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(img_left_rgb)
        plt.title(f"Left Image: {img_name_left}")
        plt.axis('off')
        plt.subplot(1, 2, 2)
        plt.imshow(img_right_rgb)
        plt.title(f"Right Image: {img_name_right}")
        plt.axis('off')
        plt.suptitle(f"Input Pair: {os.path.splitext(img_name_left)[0]} & {os.path.splitext(img_name_right)[0]}")
        plt.show()

        # 2. Compute and visualize disparity maps
        max_disp = 64  # Example: Max disparity to search (pixels). Tune this.
        block_s = 11   # Example: Block size for matching (pixels, odd number). Tune this.
        
        print(f"Computing left disparity map for {img_name_left} (max_disp={max_disp}, block_size={block_s})...")
        disparity_map_left = compute_dense_disparity_sad(img_left_gray, img_right_gray, max_disparity=max_disp, block_size=block_s)
        print(f"Computing right disparity map for {img_name_right} (max_disp={max_disp}, block_size={block_s})...")
        # For right disparity, swap images and max_disparity search range might need adjustment if non-symmetric
        disparity_map_right = compute_dense_disparity_sad(img_right_gray, img_left_gray, max_disparity=max_disp, block_size=block_s)

        plt.figure(figsize=(15, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(disparity_map_left, cmap='jet')
        plt.title(f"Left Disparity Map (D_left)")
        plt.colorbar()
        plt.axis('off')
        plt.subplot(1, 2, 2)
        plt.imshow(disparity_map_right, cmap='jet')
        plt.title(f"Right Disparity Map (D_right)")
        plt.colorbar()
        plt.axis('off')
        plt.suptitle(f"Disparity Maps for {os.path.splitext(img_name_left)[0]} & {os.path.splitext(img_name_right)[0]}")
        plt.show()

        # 3. Synthesize middle view and create GIF
        print("Synthesizing middle view...")
        # Using only left disparity map for simplicity in synthesis logic, as per common practice.
        # If using both, the synthesis function needs to be robust.
        # For the create_and_display_gif, we need img_left, img_mid, img_right.
        # The current synthesize_middle_view is a basic forward warping. 
        # It might be better to use one disparity map (e.g. left_disp) and then warp left by -d/2 and right by +d/2.
        # Let's refine synthesize_middle_view to take both images and one primary disparity map (e.g., left_to_right)
        # A more common approach for middle view synthesis:
        # img_mid(y, x - d/2) = img_left(y, x)
        # img_mid(y, x + d/2) = img_right(y, x) (assuming d is from left img perspective, so right point is x_l - d)
        # This implies we need to be careful with pixel coordinates.

        # Let's use a modified synthesis that is more common:
        # Warps left image by -D_left/2 and right image by +D_right/2 (where D_right maps right to left)
        # or more simply, warp left by -D_left/2 and right by +D_left_in_right_coords / 2
        # For now, the `synthesize_middle_view` defined earlier uses D_left for left image and optionally D_right for right image.
        # If D_right is from perspective of right image (i.e. img_right(x) maps to img_left(x - D_right(x)))
        # then for middle view: img_mid(xm) = (img_left(xm + D_left(xm)/2) + img_right(xm - D_right(xm)/2))/2
        # This is inverse warping and more complex. The current forward warping is simpler to implement.

        img_mid_synthesized = synthesize_middle_view(img_left_rgb, img_right_rgb, disparity_map_left, disparity_map_right)
        
        print("Saving temporary images for GIF...")
        cv2.imwrite(temp_image_files_for_gif[0], cv2.cvtColor(img_left_rgb, cv2.COLOR_RGB2BGR))
        cv2.imwrite(temp_image_files_for_gif[1], cv2.cvtColor(img_mid_synthesized, cv2.COLOR_RGB2BGR))
        cv2.imwrite(temp_image_files_for_gif[2], cv2.cvtColor(img_right_rgb, cv2.COLOR_RGB2BGR))
        # temp_image_files_for_gif[3] is same as [1]

        gif_filename = f"{os.path.splitext(img_name_left)[0]}_{os.path.splitext(img_name_right)[0]}_strategyA.gif"
        gif_path = os.path.join(output_dir_gifs, gif_filename)
        print(f"Creating GIF: {gif_path}")
        # The create_and_display_gif function is provided in the notebook template, ensure it's callable.
        # We need to make sure it's defined in the current scope or imported if it's in another cell.
        # For now, assuming it's available as per notebook structure.
        gif_object = create_and_display_gif(temp_image_files_for_gif, gif_path=gif_path, duration=0.5)
        display(gif_object) # display() is an IPython function for showing objects like images/GIFs
        print(f"Displayed GIF for {img_name_left} and {img_name_right}.")

    except FileNotFoundError as e:
        print(f"Error: {e}. Skipping this pair.")
    except Exception as e:
        print(f"An unexpected error occurred while processing {img_name_left} and {img_name_right} for Strategy A: {e}")
        import traceback
        traceback.print_exc()
    print("---")

print("\nFinished Part 2, Strategy A.")

# Make sure load_image is defined (it should be from Part 1, or defined here if Part 1 is not run prior)
# Ensure create_and_display_gif is defined (it's in the notebook template for this cell)
# For display(gif_object) to work, from IPython.display import display must be run.

### Explanation of your solution (short, in bullets) and interpretation of results of strategy A:

<span style="color: red; font-weight: bold;">YOUR EXPLANATION HERE</span>

In [ ]:
# ALL OF YOU CODE FOR STATEGY B SHOULD BE IN THIS CELL

# REQUIRED OUTPUTS (For each of the input stereo pairs in data/rectified_pairs):
# 1. Visualize, side-by-side: left-image | right-image (both with match-points overlaid on them - using small markers)
# 2. Visualize, side-by-side: left-image | white-image | right image (with overlayed corresponding subdivisions)
# 3. Visualize, by constructing an infinite-loop GIF of the four images: img_1->img_mid->img_2->img_mid->
 
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
import os
import imageio # Should be available from Strategy A cell or notebook template
from IPython.display import Image, display # Should be available

# Assuming load_image and compute_sift_matches are defined in Part 1 cell and available.
# If not, they need to be redefined or copied here.

def get_rectified_matches(kp1, kp2, matches, y_threshold=5, disparity_check=True):
    """Filter matches for rectified images: same y-coordinate and positive disparity."""
    rectified_matches = []
    pts1 = []
    pts2 = []
    for m in matches:
        pt1 = kp1[m.queryIdx].pt
        pt2 = kp2[m.trainIdx].pt
        if abs(pt1[1] - pt2[1]) < y_threshold: # y-coordinates should be similar
            if disparity_check:
                disparity = pt1[0] - pt2[0]
                if disparity > 0: # Disparity should be positive for typical rectified setup (pt1_x > pt2_x)
                    rectified_matches.append(m)
                    pts1.append(pt1)
                    pts2.append(pt2)
            else:
                rectified_matches.append(m)
                pts1.append(pt1)
                pts2.append(pt2)
    return rectified_matches, np.array(pts1, dtype=np.float32), np.array(pts2, dtype=np.float32)

def get_triangulation_indices(img_shape, points_np):
    """Computes Delaunay triangulation for a set of points, including image corners."""
    if points_np.shape[0] < 3:
        print("Not enough points for triangulation.")
        return np.empty((0,3), dtype=int)
    
    h, w = img_shape[:2]
    # Add image corners to the points to ensure the triangulation covers the entire image
    corner_points = np.array([
        [0, 0], [w-1, 0], [0, h-1], [w-1, h-1]
    ])
    augmented_points = np.vstack((points_np, corner_points))
    
    # Filter out duplicate points before triangulation to avoid issues
    unique_augmented_points, unique_indices = np.unique(augmented_points, axis=0, return_index=True)

    if unique_augmented_points.shape[0] < 3:
        print("Not enough unique points (after adding corners) for triangulation.")
        return np.empty((0,3), dtype=int)
        
    try:
        tri = Delaunay(unique_augmented_points)
        # The simplices refer to indices in unique_augmented_points.
        # We need to map these back to original points_np indices where possible,
        # but since we are using these triangles for warping with unique_augmented_points, it's fine.
        # The key is that tri_src_pts and tri_dst_pts in warp_triangle_affine use consistent point sets.
        return tri.simplices, unique_augmented_points # Return simplices and the points used for triangulation
    except Exception as e:
        print(f"Error during Delaunay triangulation: {e}")
        return np.empty((0,3), dtype=int), unique_augmented_points

def warp_triangle_affine(src_img, dst_img, tri_src_pts, tri_dst_pts):
    """Warps a triangle from src_img to dst_img using an affine transformation."""
    # Get the affine transform
    M = cv2.getAffineTransform(np.float32(tri_src_pts), np.float32(tri_dst_pts))
    
    # Create a mask for the destination triangle
    mask = np.zeros(dst_img.shape[:2], dtype=np.uint8)
    cv2.fillConvexPoly(mask, np.int32(tri_dst_pts), 255)
    
    # Warp the source image content
    warped_triangle = cv2.warpAffine(src_img, M, (dst_img.shape[1], dst_img.shape[0]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101)
    
    # Copy the warped triangle to the destination image using the mask
    cv2.copyTo(warped_triangle, mask, dst_img) # This modifies dst_img in place

def draw_points(image, points, color=(0, 255, 0), radius=3):
    img_copy = image.copy()
    for pt in points:
        cv2.circle(img_copy, tuple(map(int, pt)), radius, color, -1)
    return img_copy

def draw_triangles_on_image(image, points_for_triangulation, triangle_indices, color=(255, 255, 255)):
    img_with_triangles = image.copy()
    if triangle_indices.size == 0:
      return img_with_triangles
    for tri_idx in triangle_indices:
        pts = points_for_triangulation[tri_idx].astype(np.int32)
        cv2.polylines(img_with_triangles, [pts], isClosed=True, color=color, thickness=1)
    return img_with_triangles

# --- Main processing loop for Part 2, Strategy B ---
rectified_image_dir_sb = "data/rectified_pairs/" # Same dir as Strategy A
rectified_pairs_sb = [
    ("image_1.jpg", "image_2.jpg"),
    ("scene_1.jpg", "scene_2.jpg")
]

print("Starting Part 2, Strategy B: Middle-View Synthesis via Sparse Reconstruction\n")

output_dir_gifs_sb = "output_gifs_strategy_B"
if not os.path.exists(output_dir_gifs_sb):
    os.makedirs(output_dir_gifs_sb)

temp_image_files_for_gif_sb = [
    os.path.join(output_dir_gifs_sb, "_temp_frame_1_sb.png"),
    os.path.join(output_dir_gifs_sb, "_temp_frame_mid_sb.png"),
    os.path.join(output_dir_gifs_sb, "_temp_frame_2_sb.png"),
    os.path.join(output_dir_gifs_sb, "_temp_frame_mid_sb.png")
]

for img_name_left_sb, img_name_right_sb in rectified_pairs_sb:
    print(f"Processing rectified pair for Strategy B: {img_name_left_sb} and {img_name_right_sb}")
    img_left_path_sb = os.path.join(rectified_image_dir_sb, img_name_left_sb)
    img_right_path_sb = os.path.join(rectified_image_dir_sb, img_name_right_sb)

    try:
        img_left_rgb_sb, img_left_gray_sb = load_image(img_left_path_sb)
        img_right_rgb_sb, img_right_gray_sb = load_image(img_right_path_sb)

        # 1. Obtain SIFT matches (or other features)
        kp1_sb, kp2_sb, matches_sb_raw = compute_sift_matches(img_left_gray_sb, img_right_gray_sb)
        # Filter for rectified geometry
        matches_sb, pts_left_np, pts_right_np = get_rectified_matches(kp1_sb, kp2_sb, matches_sb_raw, y_threshold=5)
        print(f"Found {len(matches_sb)} good rectified SIFT matches.")

        if len(matches_sb) < 3: # Need at least 3 points for a triangle
            print("Not enough matches for triangulation. Skipping pair.")
            continue

        # 2. Determine Middle-View Point Locations
        disparities_sb = pts_left_np[:, 0] - pts_right_np[:, 0]
        pts_mid_np = np.copy(pts_left_np)
        pts_mid_np[:, 0] = pts_left_np[:, 0] - disparities_sb / 2.0

        # Visualize side-by-side: left-image | right-image (with match-points)
        img_left_with_pts = draw_points(img_left_rgb_sb, pts_left_np, color=(0,0,255))
        img_right_with_pts = draw_points(img_right_rgb_sb, pts_right_np, color=(0,0,255))
        plt.figure(figsize=(15,5))
        plt.subplot(1,2,1); plt.imshow(img_left_with_pts); plt.title(f"Left with points: {img_name_left_sb}"); plt.axis('off')
        plt.subplot(1,2,2); plt.imshow(img_right_with_pts); plt.title(f"Right with points: {img_name_right_sb}"); plt.axis('off')
        plt.suptitle("Input Images with Matched Points (Strategy B)")
        plt.show()

        # 3. Image Subdivision (Delaunay Triangulation)
        # Use pts_left_np for triangulation, then apply to pts_mid_np and pts_right_np
        # The points used for triangulation (returned_pts_for_tri) might include corner points
        triangle_indices, returned_pts_for_tri_left = get_triangulation_indices(img_left_rgb_sb.shape, pts_left_np)
        if triangle_indices.size == 0:
            print("Triangulation failed or resulted in no triangles. Skipping pair.")
            continue
        
        # Map returned_pts_for_tri_left back to mid and right points
        # This is tricky if new points (corners) were added. We need consistent point sets.
        # The simplest is to re-calculate mid and right points for the 'returned_pts_for_tri_left'
        # This assumes returned_pts_for_tri_left has its first N points as original pts_left_np
        num_original_pts = pts_left_np.shape[0]
        returned_pts_for_tri_mid = np.copy(returned_pts_for_tri_left)
        returned_pts_for_tri_right = np.copy(returned_pts_for_tri_left)

        # For original points, calculate their mid and right counterparts
        # This requires finding which of the original pts_left_np correspond to returned_pts_for_tri_left[:num_original_pts]
        # A robust way: Re-match or use an index mapping if Delaunay was on original points before adding corners.
        # For simplicity: assume first N points of returned_pts_for_tri_left are the original pts_left_np in order.
        # This is true if np.vstack was (points_np, corner_points) and points_np had no duplicates internally.
        # And if unique kept this order for the first N points.
        # A safer but slower way: for each point in returned_pts_for_tri_left, find its original match if it exists.

        # Simplified approach: Assume the first N points of returned_pts_for_tri_left match pts_left_np
        # This is only safe if pts_left_np had no internal duplicates and unique() preserved their order.
        # Let's try to make this more robust by finding the original indices.
        original_to_returned_indices = []
        for i in range(pts_left_np.shape[0]):
            # Find pts_left_np[i] in returned_pts_for_tri_left
            # This is complex due to potential floating point issues and if points were removed by unique()
            # For now, proceed with the direct mapping for points involved in triangles, assuming they are from original set.
            pass # This section needs careful handling of point correspondences if corners are added before triangulation.
                 # The current get_triangulation_indices returns simplices based on augmented_points.
                 # We need pts_mid and pts_right that correspond to these augmented_points.

        # Re-evaluate: Simpler to get triangulation on pts_left_np, then find corresponding mid/right points for *those* specific indices.
        # The `get_triangulation_indices` adds corners. We need mid/right for these corners too.
        # Corners disparity = 0 (usually). So mid points for corners are same as left points.
        disparities_aug = np.zeros(returned_pts_for_tri_left.shape[0])
        # For the original points that are part of returned_pts_for_tri_left:
        # This is still tricky. Let's assume for now that returned_pts_for_tri_left has original points first.
        # A better way: Triangulate on pts_left_np. Then, for synthesis, add 4 corner points to pts_left_np, pts_mid_np, pts_right_np.
        # And ensure triangulation uses these augmented sets from the start.
        # Let's stick to the current `get_triangulation_indices` which adds corners to input 'points_np'.
        # So returned_pts_for_tri_left = unique(vstack(pts_left_np, corners)).
        # We need to find mid and right for these.
        # For pts that were original pts_left_np, use their known disparity.
        # For corner points, disparity is 0.

        # Create mapping from returned_pts_for_tri_left to their original disparities
        # This is getting complicated. Simplification: Triangulate on original pts_left_np only for now.
        # And handle image boundaries by not warping triangles that go out of bounds too much or clip.
        # Let's retry the triangulation strategy: get indices on original pts_left_np.
        h_sb, w_sb = img_left_rgb_sb.shape[:2]
        corner_pts_sb = np.array([[0,0], [w_sb-1,0], [0,h_sb-1], [w_sb-1,h_sb-1]], dtype=np.float32)
        
        # Augment point sets for full coverage
        aug_pts_left = np.vstack((pts_left_np, corner_pts_sb))
        aug_pts_right = np.vstack((pts_right_np, corner_pts_sb)) # corners have 0 disparity relative to themselves
        aug_pts_mid = np.vstack((pts_mid_np, corner_pts_sb))   # corners have 0 disparity relative to themselves

        # Triangulate on the augmented left points
        # Need to handle potential duplicates from adding corners if some matches are already at corners
        unique_aug_pts_left, unique_idx_map = np.unique(aug_pts_left, axis=0, return_index=True)
        if unique_aug_pts_left.shape[0] < 3:
             print("Not enough unique points in augmented set for triangulation.")
             continue
        tri_sb = Delaunay(unique_aug_pts_left)
        triangle_indices_sb = tri_sb.simplices

        # Use unique_aug_pts_left for drawing and warping, and map these to corresponding mid/right points
        # The unique_idx_map tells us which original points from aug_pts_left went into unique_aug_pts_left
        final_pts_left = unique_aug_pts_left
        final_pts_mid = aug_pts_mid[unique_idx_map]
        final_pts_right = aug_pts_right[unique_idx_map]

        # Visualize subdivisions
        img_left_tri = draw_triangles_on_image(img_left_rgb_sb, final_pts_left, triangle_indices_sb)
        white_img_mid_tri = draw_triangles_on_image(np.full_like(img_left_rgb_sb, 255, dtype=np.uint8), final_pts_mid, triangle_indices_sb, color=(0,0,0))
        img_right_tri = draw_triangles_on_image(img_right_rgb_sb, final_pts_right, triangle_indices_sb)

        plt.figure(figsize=(20,6))
        plt.subplot(1,3,1); plt.imshow(img_left_tri); plt.title(f"Left with Subdivision: {img_name_left_sb}"); plt.axis('off')
        plt.subplot(1,3,2); plt.imshow(white_img_mid_tri); plt.title("Mid-points Subdivision"); plt.axis('off')
        plt.subplot(1,3,3); plt.imshow(img_right_tri); plt.title(f"Right with Subdivision: {img_name_right_sb}"); plt.axis('off')
        plt.suptitle("Image Subdivisions (Strategy B)")
        plt.show()

        # 4. Synthesize Middle View by Warping Triangles
        img_mid_synthesized_sb = np.zeros_like(img_left_rgb_sb, dtype=np.uint8)
        print(f"Warping {len(triangle_indices_sb)} triangles to synthesize middle view...")
        for tri_idx in triangle_indices_sb:
            tri_left_pts = final_pts_left[tri_idx]
            tri_mid_pts = final_pts_mid[tri_idx]
            # Warp from left image to middle view
            warp_triangle_affine(img_left_rgb_sb, img_mid_synthesized_sb, tri_left_pts, tri_mid_pts)
        # Optionally, could warp from right image for triangles on that side or blend results
        # For now, just using left image source for simplicity.

        # 5. Create and display GIF
        print("Saving temporary images for Strategy B GIF...")
        cv2.imwrite(temp_image_files_for_gif_sb[0], cv2.cvtColor(img_left_rgb_sb, cv2.COLOR_RGB2BGR))
        cv2.imwrite(temp_image_files_for_gif_sb[1], cv2.cvtColor(img_mid_synthesized_sb, cv2.COLOR_RGB2BGR))
        cv2.imwrite(temp_image_files_for_gif_sb[2], cv2.cvtColor(img_right_rgb_sb, cv2.COLOR_RGB2BGR))

        gif_filename_sb = f"{os.path.splitext(img_name_left_sb)[0]}_{os.path.splitext(img_name_right_sb)[0]}_strategyB.gif"
        gif_path_sb = os.path.join(output_dir_gifs_sb, gif_filename_sb)
        print(f"Creating Strategy B GIF: {gif_path_sb}")
        gif_object_sb = create_and_display_gif(temp_image_files_for_gif_sb, gif_path=gif_path_sb, duration=0.5)
        display(gif_object_sb)
        print(f"Displayed Strategy B GIF for {img_name_left_sb} and {img_name_right_sb}.")

    except FileNotFoundError as e:
        print(f"Error: {e}. Skipping this pair for Strategy B.")
    except Exception as e:
        print(f"An unexpected error occurred while processing {img_name_left_sb} and {img_name_right_sb} for Strategy B: {e}")
        import traceback
        traceback.print_exc()
    print("---")

print("\nFinished Part 2, Strategy B.")

# Ensure create_and_display_gif is defined (it's in the notebook template for Strategy A cell or globally)
# Ensure load_image and compute_sift_matches are available (e.g. from Part 1 cell)

### Explanation of your solution (short, in bullets) and interpretation of results of strategy B:

<span style="color: red; font-weight: bold;">YOUR EXPLANATION HERE</span>

**Strategy B: Middle-View Synthesis via Sparse Reconstruction (Triangulation & Warping)**

*   **Match Acquisition & Middle-View Point Calculation:**
    *   Explain how matches were obtained for the rectified pairs. (e.g., "Used SIFT features and filtered them for rectified geometry by checking y-coordinates and ensuring positive disparity." or "Used the general SIFT matcher from Part 1.")
    *   How were the corresponding points in the middle-view calculated? (e.g., "For each match (p_left, p_right), the disparity d = p_left.x - p_right.x was used. The middle-view point p_mid was set at (p_left.x - d/2, p_left.y).")

*   **Image Subdivision and Warping:**
    *   Describe the subdivision method. (e.g., "Used Delaunay triangulation (`scipy.spatial.Delaunay`) on the set of matched points in the left image (augmented with image corners) to create a triangular mesh.")
    *   Explain how this subdivision was transferred to the middle and right images. (e.g., "The same triangle vertex indices were used with the corresponding `pts_mid` and `pts_right` to define the mesh in the other views.")
    *   How were the triangular regions warped to synthesize the middle view? (e.g., "For each triangle in the left image, an affine transformation was computed to map it to the corresponding triangle in the middle-view. The region was then warped from the left image to the middle-view using `cv2.warpAffine` and masked appropriately.")

*   **Problems Encountered and Solutions:**
    *   **Problem:** Insufficient or poorly distributed matches leading to large, distorted triangles.
        *   **Solution Attempt:** (e.g., "This is a key challenge. Adding image corners to the point set for triangulation helped ensure the entire image area was covered, but internal regions still depend on match density. More robust feature matching or denser matching could improve this.")
    *   **Problem:** Artifacts at triangle boundaries or within triangles.
        *   **Solution Attempt:** (e.g., "Affine warping assumes planar regions. If this assumption is violated, or if matches are not perfectly accurate, visual artifacts can appear. Using smaller triangles (more matches) can help. Blending at triangle edges could smooth transitions but was not implemented.")
    *   **Problem:** Handling occlusions or disocclusions with sparse features.
        *   **Solution Attempt:** (e.g., "This method inherently struggles with occlusions if not captured by the sparse features. The warped triangles might 'stretch' over occluded areas or use information from only one view. The strategy primarily relies on the geometry defined by the matched points.")
    *   **Problem:** Selecting which image (left or right) to warp from for each triangle, or how to blend.
        *   **Solution Attempt:** (e.g., "The current implementation warped from the left image. A more advanced approach could warp from both and blend, potentially weighting by view angle or distance, or choosing the view with less distortion for that triangle.")

*   **Interpretation of Results (Visualizations and GIF):**
    *   Comment on the quality of the match visualizations and the subdivisions.
    *   Assess the visual quality of the synthesized middle view and the GIF animation. (e.g., "The GIF showed that the method can produce plausible middle views, especially where match density is good. However, distortions were evident in sparsely matched regions. The triangulation overlay clearly showed how the scene was parameterized.")
    *   How does this compare to Strategy A? (e.g., "Strategy B might look more 'piecewise correct' due to geometric warping but can suffer from insufficient features, while Strategy A provides denser results but can have 'wavy' artifacts from local disparity errors.")

**Note to Student:** Elaborate on these points based on your actual implementation and the visual results.

# Part 3 - Motion Segmentation
---

### In this section, you are given a pair of images, taken from two different viewpoints, at different times. The captured scene is known to be static, except for a single rigid (non-flexible) object that has moved between the two  captures. Your goal is to 'segment' (classify) the set of matches between the images, as belonging to either 'static' (background) or 'dynamic' (moving object) or 'outliers' (inconsistent).

## Instructions
* Use only the functions that you had implemented in part 1
* Show results for the image pair under the 'data/pairs_to_segment' folder
* Capture yourselves two additional image pairs, save them in that same folder, and produce results for them too.


## Required Visualization (for each image pair):
-- See the 'segmentation' solution outputs under the 'examples' folder
* Show the image pair, with 'outlier' matches overlaid (show only a random subset if there are too many)
* Show the image pair, with 'static' matches overlaid (show only a random subset if there are too many)
* Show the image pair, with 'dynamic' matches overlaid (show only a random subset if there are too many

## Implement your solution in the next cell and explain it in short in the following one.


In [ ]:
# ALL OF YOUR CODE FOR PART 3 SHOULD BE IN THIS CELL

# REQUIRED OUTPUTS (For each of the input stereo pairs in data/pairs_to_segment):
# 1. Visualize outlier matches
# 2. Visualize static matches (background)
# 3. Visualize dynamic matches (moving object)

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Assuming load_image, compute_sift_matches, estimate_fundamental_matrix, 
# and visualize_matches are defined in Part 1 cell and available in the notebook's scope.

def segment_motion(img1_gray, img2_gray, kp1, kp2, matches, min_matches_for_model=10):
    """Segments matches into static, dynamic, and outliers."""
    static_matches = []
    dynamic_matches = []
    outlier_matches = []

    if len(matches) < min_matches_for_model: # Need enough matches to even attempt F_bg
        print(f"Not enough initial matches ({len(matches)}) to perform segmentation. Classifying all as outliers.")
        return [], [], matches

    # 1. Estimate Background Model (F_bg)
    F_bg, mask_bg_indices = estimate_fundamental_matrix(kp1, kp2, matches)
    
    matches_non_static_candidates = []
    if F_bg is not None and mask_bg_indices is not None:
        mask_bg_flat = mask_bg_indices.ravel().astype(bool)
        for i, match in enumerate(matches):
            if mask_bg_flat[i]:
                static_matches.append(match)
            else:
                matches_non_static_candidates.append(match)
        print(f"Background model F_bg estimated. Found {len(static_matches)} static matches.")
    else:
        print("Could not estimate background F_bg. Treating all matches as potential non-static.")
        matches_non_static_candidates = list(matches) # Treat all as non-static if F_bg fails

    if not matches_non_static_candidates or len(matches_non_static_candidates) < min_matches_for_model:
        print(f"Not enough non-static matches ({len(matches_non_static_candidates)}) for object motion estimation. Remaining are outliers.")
        outlier_matches = matches_non_static_candidates
        return static_matches, [], outlier_matches

    # 2. Estimate Object Motion Model (H_obj) from non-static matches
    # Extract points for Homography estimation
    pts_obj_cand_img1 = np.float32([kp1[m.queryIdx].pt for m in matches_non_static_candidates]).reshape(-1, 1, 2)
    pts_obj_cand_img2 = np.float32([kp2[m.trainIdx].pt for m in matches_non_static_candidates]).reshape(-1, 1, 2)

    if len(pts_obj_cand_img1) >= 4: # Need at least 4 points for findHomography
        H_obj, mask_obj_indices = cv2.findHomography(pts_obj_cand_img1, pts_obj_cand_img2, cv2.RANSAC, 5.0)
        if H_obj is not None and mask_obj_indices is not None:
            mask_obj_flat = mask_obj_indices.ravel().astype(bool)
            for i, match in enumerate(matches_non_static_candidates):
                if mask_obj_flat[i]:
                    dynamic_matches.append(match)
                else:
                    outlier_matches.append(match) # Not fitting F_bg nor H_obj
            print(f"Object model H_obj estimated. Found {len(dynamic_matches)} dynamic matches.")
        else:
            print("Could not estimate object H_obj. Classifying remaining non-static matches as outliers.")
            outlier_matches = matches_non_static_candidates
    else:
        print(f"Not enough non-static candidate points ({len(pts_obj_cand_img1)}) for Homography. Classifying all as outliers.")
        outlier_matches = matches_non_static_candidates
        
    print(f"Segmentation complete: Static={len(static_matches)}, Dynamic={len(dynamic_matches)}, Outliers={len(outlier_matches)}")
    return static_matches, dynamic_matches, outlier_matches


# --- Main processing loop for Part 3 ---
segment_image_dir = "data/pairs_to_segment/"
segment_image_pairs = [
    ("tuna_1.jpg", "tuna_2.jpg"),
    # Placeholders for user's two additional pairs
    ("user_pair1_img1.jpg", "user_pair1_img2.jpg"), 
    ("user_pair2_img1.jpg", "user_pair2_img2.jpg")
]

print("Starting Part 3: Motion Segmentation\n")

for img_name1_seg, img_name2_seg in segment_image_pairs:
    print(f"Processing pair for segmentation: {img_name1_seg} and {img_name2_seg}")
    img1_path_seg = os.path.join(segment_image_dir, img_name1_seg)
    img2_path_seg = os.path.join(segment_image_dir, img_name2_seg)

    if not os.path.exists(img1_path_seg) or not os.path.exists(img2_path_seg):
        print(f"Image pair not found: {img1_path_seg} or {img2_path_seg}. Skipping this pair.")
        print("Ensure you have added your custom pairs to the 'data/pairs_to_segment/' folder if these are user pairs.")
        print("---")
        continue

    try:
        img1_rgb_seg, img1_gray_seg = load_image(img1_path_seg)
        img2_rgb_seg, img2_gray_seg = load_image(img2_path_seg)

        kp1_seg, kp2_seg, matches_initial_seg = compute_sift_matches(img1_gray_seg, img2_gray_seg)
        print(f"Found {len(matches_initial_seg)} initial SIFT matches for {img_name1_seg}-{img_name2_seg}.")

        if not matches_initial_seg:
            print("No SIFT matches found. Cannot perform motion segmentation.")
            print("---")
            continue

        static_m, dynamic_m, outlier_m = segment_motion(img1_gray_seg, img2_gray_seg, kp1_seg, kp2_seg, matches_initial_seg)

        # Visualize Outlier Matches
        if outlier_m:
            title_outliers = f"Outlier Matches: {os.path.splitext(img_name1_seg)[0]} & {os.path.splitext(img_name2_seg)[0]}"
            visualize_matches(img1_rgb_seg, kp1_seg, img2_rgb_seg, kp2_seg, outlier_m, title=title_outliers)
        else:
            print("No outlier matches to visualize.")

        # Visualize Static Matches (Background)
        if static_m:
            title_static = f"Static Matches (Background): {os.path.splitext(img_name1_seg)[0]} & {os.path.splitext(img_name2_seg)[0]}"
            visualize_matches(img1_rgb_seg, kp1_seg, img2_rgb_seg, kp2_seg, static_m, title=title_static)
        else:
            print("No static matches to visualize.")

        # Visualize Dynamic Matches (Moving Object)
        if dynamic_m:
            title_dynamic = f"Dynamic Matches (Moving Object): {os.path.splitext(img_name1_seg)[0]} & {os.path.splitext(img_name2_seg)[0]}"
            visualize_matches(img1_rgb_seg, kp1_seg, img2_rgb_seg, kp2_seg, dynamic_m, title=title_dynamic)
        else:
            print("No dynamic matches to visualize.")

    except FileNotFoundError:
        # This case should be caught by the os.path.exists check earlier, but as a safeguard:
        print(f"Error: Image file not found for pair {img_name1_seg}, {img_name2_seg}. Skipping.")
    except Exception as e:
        print(f"An unexpected error occurred while processing {img_name1_seg} and {img_name2_seg} for Part 3: {e}")
        import traceback
        traceback.print_exc()
    print("---")

print("\nFinished Part 3.")

### Explanation of your solution (short, in bullets) and interpretation of your results.

<span style="color: red; font-weight: bold;">YOUR EXPLANATION HERE</span>

**Part 3: Motion Segmentation**

*   **Overall Approach:**
    *   Briefly describe your strategy for segmenting matches into static, dynamic, and outliers. (e.g., "The approach involved first identifying static background matches using a Fundamental matrix (F_bg). Matches not fitting this model were then analyzed to find a consistent motion model (Homography H_obj) for the dynamic object. Remaining matches were considered outliers.")
    *   Mention the use of functions from Part 1 (e.g., SIFT matching, F-matrix estimation, match visualization).

*   **Static Matches (Background):**
    *   How was the Fundamental matrix `F_bg` for the background estimated? (e.g., "Computed SIFT matches between the image pair. Then, `cv2.findFundamentalMat` with RANSAC was used to estimate `F_bg` from all matches, assuming the background constitutes the dominant motion.")
    *   How were matches classified as static? (e.g., "Matches that were inliers to `F_bg` (based on the RANSAC inlier mask) were classified as static background points.")

*   **Dynamic Matches (Moving Object):**
    *   How were potential dynamic matches isolated? (e.g., "Matches that were not inliers to `F_bg` were considered potential dynamic matches or outliers.")
    *   What model was used for the moving object? (e.g., "A Homography matrix `H_obj` was estimated from these non-static matches using `cv2.findHomography` with RANSAC. This assumes the moving object is planar or its motion can be approximated by a homography.")
    *   How were matches classified as dynamic? (e.g., "Non-static matches that were inliers to `H_obj` were classified as dynamic (belonging to the moving object).")
    *   What happened if an object motion model (`H_obj`) couldn't be robustly found? (e.g., "If too few non-static points existed or RANSAC failed to find a consistent `H_obj`, these points might primarily fall into the 'outlier' category by default, or an alternative strategy might be needed.")

*   **Outlier Matches:**
    *   How were outliers finally determined? (e.g., "Matches that did not fit the background model `F_bg` and also did not fit the object motion model `H_obj` were classified as outliers.")

*   **Interpretation of Results (Visualizations):**
    *   For each image pair processed (the provided 'tuna' pair and your two custom pairs):
        *   Discuss the effectiveness of the segmentation. Were static, dynamic, and outlier matches correctly identified for the most part?
        *   Comment on the visualizations. Did they clearly show the different categories of matches?
        *   Were there any challenges specific to any image pair? (e.g., "For the 'tuna' pair, the segmentation worked well in distinguishing the fish's motion from the static background." or "For user_pair1, the small size of the object made robust H_obj estimation difficult, leading to some dynamic points being misclassified as outliers.")
        *   Mention the number of matches in each category if insightful.

*   **Regarding the Two Custom Image Pairs:**
    *   Briefly describe the two image pairs you captured/found (object, type of motion/rotation).
    *   Confirm they were added to the `data/pairs_to_segment/` folder.
    *   Discuss how well your algorithm performed on them.

**Note to Student:** Elaborate on these points based on your actual implementation and the visual results from running the code on all three pairs.